In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')


In [2]:
#insert app_train data

In [3]:
print('Training data shape: ', app_train.shape)
app_train.head()

NameError: name 'app_train' is not defined

In [ ]:
#insert app_test data

In [ ]:
print('Testing data shape: ', app_test.shape)
app_test.head()

In [ ]:
# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

In [ ]:
# one-hot encoding of categorical variables
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

In [ ]:
train_labels = app_train['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

# Add the target back in
app_train['TARGET'] = train_labels

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler, Imputer

# Drop the target from the training data
if 'TARGET' in app_train:
    train = app_train.drop(columns = ['TARGET'])
else:
    train = app_train.copy()
    
# Feature names
features = list(train.columns)

# Copy of the testing data
test = app_test.copy()

# Median imputation of missing values
imputer = Imputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

# Fit on the training data
imputer.fit(train)

# Transform both training and testing data
train = imputer.transform(train)
test = imputer.transform(app_test)

# Repeat with the scaler
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001)

# Train on the training data
log_reg.fit(train, train_labels)

In [ ]:
# Make predictions
# Make sure to select the second column only
log_reg_pred = log_reg.predict_proba(test)[:, 1]
print(log_reg_pred)

In [9]:
import urllib3, requests, json

# retrieve your wml_service_credentials_username, wml_service_credentials_password, and wml_service_credentials_url from the
# Service credentials associated with your IBM Cloud Watson Machine Learning Service instance
wml_service_credentials_username = "9cd73f73-1bc4-4695-9b87-2b0bcebd6b9f"
wml_service_credentials_password = "935833ca-8838-4d08-8679-1a87e267b12e"
wml_service_credentials_url = "https://ibm-watson-ml.eu-gb.bluemix.net"
wml_credentials={
"url": wml_service_credentials_url,
"username": wml_service_credentials_username,
"password": wml_service_credentials_password
}

headers = urllib3.util.make_headers(basic_auth='{username}:{password}'.format(username=wml_credentials['username'], password=wml_credentials['password']))
url = '{}/v3/identity/token'.format(wml_credentials['url'])
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}
array_of_values_to_be_scored=[
       1,
      1000,
      1,
      1,
      2,
      45,
      1,
      1,
      0,
      -1,
      -1,
      0,
      1000,
      1000,
      1000,
      1000,
      1000,
      1000,
      1000,
      1000,
      1000,
      1000,
      1000,
      1000]

another_array_of_values_to_be_scored=[ 1,
      1000,
      1,
      1,
      2,
      45,
      1,
      1,
      0,
      -1,
      -1,
      0,
      1000,
      1000,
      1000,
      1000,
      1000,
      1000,
      1000,
      1000,
      1000,
      1000,
      1000,
      1000]
# NOTE: manually define and pass the array(s) of values to be scored in the next line
payload_scoring = {"fields": ["ID", "LIMIT_BAL", "SEX", "EDUCATION", "MARRIAGE", "AGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6", "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"], 
                   "values": [array_of_values_to_be_scored, another_array_of_values_to_be_scored]}

response_scoring = requests.post('https://ibm-watson-ml.eu-gb.bluemix.net/v3/wml_instances/8eb67f5f-a531-4693-99e8-f3ca7773e5dc/deployments/e5f96e0f-3ec5-4104-9d7a-bf4524eaf19b/online', json=payload_scoring, headers=header)
print("Scoring response")
print(json.loads(response_scoring.text))


 Scoring response
{'fields': ['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'features', 'rawPrediction', 'probability', 'prediction', 'nodeADP_class', 'nodeADP_classes'], 'values': [[1, 1000, 1, 1, 2, 45, 1, 1, 0, -1, -1, 0, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, [0.00011503968227135422, 0.007709539662191428, 2.0457996460399994, 1.2642367813215571, 3.831132075344397, 4.884034679262907, 0.8842975631736183, 0.8331071821743725, 0.0, -0.8592202333921024, -0.8857977160325888, 0.0, 0.01349864129059024, 0.014002011469540138, 0.01429218174953399, 0.01541872501766167, 0.016386852338615445, 0.016771296626639745, 0.060167662256913945, 0.04251526808113944, 0.0562548029411415, 0.06679033373968925, 0.06361700109215314, 0.05629926051908366], [0.3344156515